# Universidad Del Valle de Guatemala
## Deep Learning
### Proyecto Final: Red Neuronal Recurrente que predice el ganador de un partido de la liga BBVA.
#### Integrantes: 
- Roberto Rios 20979
- Javier Mombiela 20067

#### Importación de librerias

In [72]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.layers import Dense, Embedding, LSTM, Dropout, RNN

#### Carga de data

In [73]:
data = pd.read_csv('temporadas.csv')

Visualizacion de data

In [74]:
print(data.count())
data

Date        3800
HomeTeam    3800
AwayTeam    3800
FTHG        3800
FTAG        3800
FTR         3800
HTHG        3800
HTAG        3800
HTR         3800
HS          3800
AS          3800
HST         3800
AST         3800
HF          3800
AF          3800
HC          3800
AC          3800
HY          3800
AY          3800
HR          3800
AR          3800
dtype: int64


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,17/08/13,Sociedad,Getafe,2,0,H,1,0,H,16,...,6,2,13,6,6,5,1,1,0,0
1,17/08/13,Valencia,Malaga,1,0,H,0,0,D,9,...,1,2,15,23,9,6,3,5,0,0
2,17/08/13,Valladolid,Ath Bilbao,1,2,A,1,1,D,8,...,2,3,10,8,5,5,1,0,0,0
3,18/08/13,Barcelona,Levante,7,0,H,6,0,H,22,...,13,1,15,16,9,3,1,3,0,0
4,18/08/13,Osasuna,Granada,1,2,A,0,2,A,14,...,5,4,15,17,7,6,1,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,04/06/2023,Betis,Valencia,1,1,D,1,0,H,8,...,3,3,11,11,5,4,3,0,0,1
3796,04/06/2023,Celta,Barcelona,2,1,H,1,0,H,12,...,9,2,13,9,2,4,2,2,0,0
3797,04/06/2023,Elche,Cadiz,1,1,D,0,1,A,12,...,5,3,18,8,7,4,4,3,0,0
3798,04/06/2023,Espanol,Almeria,3,3,D,1,1,D,12,...,8,4,11,12,7,6,2,3,0,0


Comprobar si hay valores nulos o faltantes en el conjunto de datos

In [75]:
print(data.isnull().sum())

Date        0
HomeTeam    0
AwayTeam    0
FTHG        0
FTAG        0
FTR         0
HTHG        0
HTAG        0
HTR         0
HS          0
AS          0
HST         0
AST         0
HF          0
AF          0
HC          0
AC          0
HY          0
AY          0
HR          0
AR          0
dtype: int64


#### Preprocesamiento de datos

Codificacion de etiquetas para los equipos y variable objeitov FTR (Full Time Result)

In [76]:
team_label_encoder = LabelEncoder()
data['HomeTeam'] = team_label_encoder.fit_transform(data['HomeTeam'])
data['AwayTeam'] = team_label_encoder.fit_transform(data['AwayTeam'])

result_label_encoder = LabelEncoder()
data['FTR'] = result_label_encoder.fit_transform(data['FTR'])

Ver la codificacion de cada variable

In [77]:
team_mapping = dict(zip(team_label_encoder.classes_, team_label_encoder.transform(team_label_encoder.classes_)))
print(team_mapping)

{'Alaves': 0, 'Almeria': 1, 'Ath Bilbao': 2, 'Ath Madrid': 3, 'Barcelona': 4, 'Betis': 5, 'Cadiz': 6, 'Celta': 7, 'Cordoba': 8, 'Eibar': 9, 'Elche': 10, 'Espanol': 11, 'Getafe': 12, 'Girona': 13, 'Granada': 14, 'Huesca': 15, 'La Coruna': 16, 'Las Palmas': 17, 'Leganes': 18, 'Levante': 19, 'Malaga': 20, 'Mallorca': 21, 'Osasuna': 22, 'Real Madrid': 23, 'Sevilla': 24, 'Sociedad': 25, 'Sp Gijon': 26, 'Valencia': 27, 'Valladolid': 28, 'Vallecano': 29, 'Villarreal': 30}


In [78]:
result_mapping = dict(zip(result_label_encoder.classes_, result_label_encoder.transform(result_label_encoder.classes_)))
print(result_mapping)

{'A': 0, 'D': 1, 'H': 2}


Visualizar las primeras filas para comprobar la codificacion

In [79]:
data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,17/08/13,25,12,2,0,2,1,0,H,16,...,6,2,13,6,6,5,1,1,0,0
1,17/08/13,27,20,1,0,2,0,0,D,9,...,1,2,15,23,9,6,3,5,0,0
2,17/08/13,28,2,1,2,0,1,1,D,8,...,2,3,10,8,5,5,1,0,0,0
3,18/08/13,4,19,7,0,2,6,0,H,22,...,13,1,15,16,9,3,1,3,0,0
4,18/08/13,22,14,1,2,0,0,2,A,14,...,5,4,15,17,7,6,1,4,0,0


#### Agregar estadisticas a la data

Funciones lambda para calcular estadisticas (puntos y diferencia de goles de los ultimos 10 partidos)

In [80]:
data['PH'] = data.apply(lambda row: 3 if row['FTR'] == 2 else 1 if row['FTR'] == 0 else 0, axis=1)
data['PA'] = data.apply(lambda row: 3 if row['FTR'] == 1 else 1 if row['FTR'] == 0 else 0, axis=1)
data['GDH'] = data['FTHG'] - data['FTAG']
data['GDA'] = data['FTAG'] - data['FTHG']

Calculos de los ultimos 10 partidos

In [81]:
for team in data['HomeTeam'].unique():
    mask = (data['HomeTeam'] == team)
    data.loc[mask, 'PH'] = data.loc[mask, 'PH'].rolling(10).sum()
    data.loc[mask, 'GDH'] = data.loc[mask, 'GDH'].rolling(10).sum()

for team in data['AwayTeam'].unique():
    mask = (data['AwayTeam'] == team)
    data.loc[mask, 'PA'] = data.loc[mask, 'PA'].rolling(10).sum()
    data.loc[mask, 'GDA'] = data.loc[mask, 'GDA'].rolling(10).sum()

Colaborar que si se hayan agregado las nuevas columnas al dataset

In [82]:
data.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,HC,AC,HY,AY,HR,AR,PH,PA,GDH,GDA
3795,04/06/2023,5,27,1,1,1,1,0,H,8,...,5,4,3,0,0,1,15.0,5.0,2.0,-6.0
3796,04/06/2023,7,4,2,1,2,1,0,H,12,...,2,4,2,2,0,0,18.0,8.0,6.0,4.0
3797,04/06/2023,10,6,1,1,1,0,1,A,12,...,7,4,4,3,0,0,13.0,13.0,-1.0,-10.0
3798,04/06/2023,11,1,3,3,1,1,1,D,12,...,7,6,2,3,0,0,10.0,7.0,-4.0,-12.0
3799,04/06/2023,28,12,0,0,1,0,0,D,15,...,6,1,1,3,0,0,15.0,13.0,-3.0,-6.0


Colaborar que la suma de los puntos sea la correcta (verifiamos con los ultimos 15 partidos de un equipo)

In [86]:
team = 4 # Barcelona con codigo 4

matches = data[(data['HomeTeam'] == team) | (data['AwayTeam'] == team)]

# obtener los ultimos 15 partidos
matches.tail(15)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,HC,AC,HY,AY,HR,AR,PH,PA,GDH,GDA
3656,05/03/2023,4,27,1,0,2,1,0,H,13,...,5,2,0,0,1,0,27.0,10.0,20.0,-6.0
3668,12/03/2023,2,4,0,1,0,0,1,A,15,...,5,3,2,1,0,0,19.0,8.0,7.0,5.0
3679,19/03/2023,4,23,2,1,2,1,1,D,17,...,8,1,4,3,0,0,27.0,8.0,18.0,6.0
3684,01/04/2023,10,4,0,4,0,0,1,A,15,...,11,3,0,3,0,0,10.0,8.0,-10.0,8.0
3699,10/04/2023,4,13,0,0,1,0,0,D,18,...,12,2,0,2,0,0,24.0,14.0,17.0,-3.0
3706,16/04/2023,12,4,0,0,1,0,0,D,5,...,1,5,2,1,0,0,14.0,11.0,4.0,10.0
3717,23/04/2023,4,3,1,0,2,1,0,H,13,...,3,3,4,7,0,0,24.0,11.0,15.0,2.0
3726,26/04/2023,29,4,2,1,2,1,0,H,9,...,3,9,1,4,0,0,12.0,10.0,0.0,8.0
3733,29/04/2023,4,5,4,0,2,3,0,H,15,...,10,0,0,2,0,1,24.0,7.0,15.0,-6.0
3741,02/05/2023,4,22,1,0,2,0,0,D,21,...,11,0,0,0,0,1,24.0,17.0,14.0,-1.0


#### Procesamiento de datos